In [1]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import imdb
import time
import json
import ast
import shutil
import numpy as np
import math
pd.options.display.max_rows=400
pd.options.display.max_columns=40
pd.options.display.max_colwidth=100

# 1 Extracting Name, Location, Year # with filename that we have

In [3]:
Movie_files = []
rootdir = 'G:\\'
for dirpath, subdirs, files in os.walk(rootdir):
    for x in files:
        if x.endswith((".mkv",".mp4",".avi")) :
            Movie_files.append(os.path.join(dirpath, x))
            
Movie_files

['G:\\0 Director\\Abdellatif Kechiche\\Blue Is the Warmest Color (2013) Tigole 7.4 3000\\Blue_Is_the_Warmest_Color_2013_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Bandi_30NAMA.mkv',
 'G:\\0 Director\\Abdellatif Kechiche\\The Secret Of The Grain (2007) Tigole 7.5 300\\The_Secret_of_the_Grain_2007_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Tigole_30NAMA.mkv',
 'G:\\0 Director\\Abdellatif Kechiche\\Vénus noire (2010) dvd 7 100\\Black_Venus_2010_CD1_DVDRip_RedBlade_30NAMA.avi',
 'G:\\0 Director\\Abdellatif Kechiche\\Vénus noire (2010) dvd 7 100\\Black_Venus_2010_CD2_DVDRip_RedBlade_30NAMA.avi',
 'G:\\0 Director\\Alejandro Inarritu\\21 Grams (2003) Tigole 6.5\\21 Grams (2003) (1080p BluRay x265 afm72).mkv',
 'G:\\0 Director\\Alejandro Inarritu\\Amores Perros (2000) Tigole 7.6\\Amores Perros (2000) (1080p BluRay x265 Bandi).mkv',
 'G:\\0 Director\\Alejandro Inarritu\\Amores Perros (2000) Tigole 7.6\\Featurettes\\Deleted scenes (with optional commentary by Inarritu and Prieto).mkv',
 'G:\

In [4]:
Movie_files_series=pd.Series(Movie_files)
Movie_files_series

0       G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000\Blue_Is_the_W...
1       G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300\The_Secret_of_th...
2       G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100\Black_Venus_2010_CD1_DVDRip_RedBl...
3       G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100\Black_Venus_2010_CD2_DVDRip_RedBl...
4       G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5\21 Grams (2003) (1080p BluRay x265 a...
                                                       ...                                                 
1953                              G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes\Behind the Scenes.mkv
1954                                         G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes\Teaser.mkv
1955                                        G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes\Trailer.mkv
1956    G:\2015 - 2020\cemet

In [5]:
splited_path=Movie_files_series.apply(lambda x:os.path.split(x)[0])
file_name=Movie_files_series.apply(lambda x:os.path.split(x)[1])
print(splited_path)
print(file_name)

0       G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000
1          G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300
2                           G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100
3                           G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100
4                              G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5
                                               ...                                        
1953                                   G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes
1954                                   G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes
1955                                   G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes
1956              G:\2015 - 2020\cemetery.of.splendor.2015.limited.1080p.bluray.x264-depth
1957                                                G:\2015 - 2020\Paterson (2016) Silence

In [102]:
# def path_extractor(Movie_files):
#     splited_path=[]
#     file_name=[]
#     for path in Movie_files:
#         spliting=os.path.split(path)
#         splited_path.append(spliting[0])
#         file_name.append(spliting[1])

### regex for movies

In [6]:
regex_final=r".+?([\(\._]?\d{4}\)?)(?=[\.+\)\s_])"

In [7]:
list_of_movies=file_name.apply(lambda x: re.search(regex_final,x).group(0) if re.search(regex_final,x) is not None else np.nan)
list_of_movies

0       Blue_Is_the_Warmest_Color_2013
1         The_Secret_of_the_Grain_2007
2                     Black_Venus_2010
3                     Black_Venus_2010
4                      21 Grams (2003)
                     ...              
1953                               NaN
1954                               NaN
1955                               NaN
1956         Cemetery.of.Splendor.2015
1957                   Paterson (2016)
Length: 1958, dtype: object

In [8]:
list_of_movies=list_of_movies.str.replace('_',' ').str.replace('.',' ').str.replace(r'(.+?)(?<=\s)(\d{4})',repl=lambda x: x.group(1)+"({})".format(x.group(2)), regex=True)
list_of_movies

C:\Users\zolfaghari\AppData\Local\Temp\ipykernel_17680\3732611098.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  list_of_movies=list_of_movies.str.replace('_',' ').str.replace('.',' ').str.replace(r'(.+?)(?<=\s)(\d{4})',repl=lambda x: x.group(1)+"({})".format(x.group(2)), regex=True)


0       Blue Is the Warmest Color (2013)
1         The Secret of the Grain (2007)
2                     Black Venus (2010)
3                     Black Venus (2010)
4                        21 Grams (2003)
                      ...               
1953                                 NaN
1954                                 NaN
1955                                 NaN
1956         Cemetery of Splendor (2015)
1957                     Paterson (2016)
Length: 1958, dtype: object

In [9]:
df1=pd.DataFrame(list_of_movies)

In [11]:
df1.rename(columns={0:'Title'},inplace=True)
separated_path=pd.concat([df1,splited_path,file_name],axis=1)

In [12]:
separated_path.rename(columns={0:'Folder_Path', 1:'File_Path'},inplace=True)
separated_path

,Title,Folder_Path,File_Path
0,Blue Is the Warmest Color (2013),G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000,Blue_Is_the_Warmest_Color_2013_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Bandi_30NAMA.mkv
1,The Secret of the Grain (2007),G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300,The_Secret_of_the_Grain_2007_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Tigole_30NAMA.mkv
2,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD1_DVDRip_RedBlade_30NAMA.avi
3,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD2_DVDRip_RedBlade_30NAMA.avi
4,21 Grams (2003),G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5,21 Grams (2003) (1080p BluRay x265 afm72).mkv
...,...,...,...
1953,NaN,G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes,Behind the Scenes.mkv
1954,NaN,G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes,Teaser.mkv
1955,NaN,G:\2015 - 2020\Burning (2018) 4K Tigole\Featurettes,Trailer.mkv
1956,Cemetery of Splendor (2015),G:\2015 - 2020\cemetery.of.splendor.2015.limited.1080p.bluray.x264-depth,Cemetery.of.Splendor.2015.LIMITED.1080p.BluRay.x264-DEPTH.mkv


### Remove extras

In [13]:
extrs_indexes=separated_path.loc[separated_path.Folder_Path.str.contains('Extras|Featurettes')].index
extrs_indexes

Int64Index([   6,    7,    8,    9,   10,   11,   12,   13,   14,   15,
            ...
            1926, 1931, 1932, 1933, 1934, 1937, 1952, 1953, 1954, 1955],
           dtype='int64', length=1106)

In [ ]:
final.loc[final.Title.isna()]

In [14]:
separated_path.drop(extrs_indexes,inplace=True)
separated_path.dropna(axis=0,inplace=True)
final=separated_path.reset_index()
final.drop(columns=['index'],inplace=True)
final.drop_duplicates(subset='Title', inplace=True)
final.reset_index(drop=True,inplace=True)
final

,Title,Folder_Path,File_Path
0,Blue Is the Warmest Color (2013),G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000,Blue_Is_the_Warmest_Color_2013_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Bandi_30NAMA.mkv
1,The Secret of the Grain (2007),G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300,The_Secret_of_the_Grain_2007_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Tigole_30NAMA.mkv
2,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD1_DVDRip_RedBlade_30NAMA.avi
3,21 Grams (2003),G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5,21 Grams (2003) (1080p BluRay x265 afm72).mkv
4,Amores Perros (2000),G:\0 Director\Alejandro Inarritu\Amores Perros (2000) Tigole 7.6,Amores Perros (2000) (1080p BluRay x265 Bandi).mkv
...,...,...,...
767,Happy Hour (2015),G:\2015 - 2020\8 - 9\6 Happy Hour (2015) VXT 8.5 100,Happy Hour 2015 JAPANESE 720p BluRay H264 AAC-VXT.mkv
768,Dominion (2018),G:\2015 - 2020\8 - 9\7 Dominion (2018) YTS 8.6 0,Dominion_2018_1080p_WEB-DL_30nama_30NAMA.mkv
769,colliding dreams (2015),G:\2015 - 2020\8 - 9\8 Colliding Dreams (2015) dvd 8 0,colliding.dreams.2015.limited.dvdrip.x264.mkv
770,Cemetery of Splendor (2015),G:\2015 - 2020\cemetery.of.splendor.2015.limited.1080p.bluray.x264-depth,Cemetery.of.Splendor.2015.LIMITED.1080p.BluRay.x264-DEPTH.mkv


In [15]:
final.to_csv('hard10.csv',index=False)

### extract encoder
##### list of encoders from 30nama and use of power automate

In [265]:
final=pd.read_csv('hard10.csv')
final

,Title,Folder_Path,File_Path
0,Blue Is the Warmest Color (2013),G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000,Blue_Is_the_Warmest_Color_2013_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Bandi_30NAMA.mkv
1,The Secret of the Grain (2007),G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300,The_Secret_of_the_Grain_2007_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Tigole_30NAMA.mkv
2,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD1_DVDRip_RedBlade_30NAMA.avi
3,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD2_DVDRip_RedBlade_30NAMA.avi
4,21 Grams (2003),G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5,21 Grams (2003) (1080p BluRay x265 afm72).mkv
...,...,...,...
1585,Dominion (2018),G:\2015 - 2020\8 - 9\7 Dominion (2018) YTS 8.6 0,Dominion_2018_1080p_WEB-DL_30nama_30NAMA.mkv
1586,colliding dreams (2015),G:\2015 - 2020\8 - 9\8 Colliding Dreams (2015) dvd 8 0,colliding.dreams.2015.limited.dvdrip.x264.mkv
1587,Burning (2018),G:\2015 - 2020\Burning (2018) 4K Tigole,Burning (2018) (2160p BluRay x265 10bit HDR Tigole).mkv
1588,Cemetery of Splendor (2015),G:\2015 - 2020\cemetery.of.splendor.2015.limited.1080p.bluray.x264-depth,Cemetery.of.Splendor.2015.LIMITED.1080p.BluRay.x264-DEPTH.mkv


In [16]:
encoders1=['30nama','YIFY','PaHe','RMTeam','PSA','RARBG','Ohys-Raws','HorribleSubs','Judas','AnimeRG','ASW','d3g','Tigole','AdiT','Erai-raws','Cleo','Phanteam','Ganool','MANiC','VXT','MkvCage','SS','EVO','LoveBug','DB','mSD','RCVR','NoGrouP','BTN','NoGrp','squalor','SAiNTS','MeGusta','NTb','CtrlHD','SubsPlease','Kirion','Goki','TVV','SA89','WELP','NODLABS','Vyndros','SNAKE','aAF','OSiTV','pseudo','DON','Sotark','Anime-Time','Yakubo','jlw','afm72','x0r','ViKi','EMBER','QOQ','Telly','AJP69','bonkai77','Reaktor','CMRG','HANDJOB','NTG','FGT','HoneyG','DEiMOS','GalaxyTV','Will1869','PHOCiS','zza','JiTB','iFT','MRN','MZABI','TEPES','ETRG','FFNDVD','Ravarage','SFM','ImE','ShAaNiG','Mana','GalaxyRG','PLAiD','WeTv','fov','CasStudio','Garshasp','AppleTor','KRaLiMaRKo','HDMaNiAcS','BTW','IPT','DarkDream','t3nzin','NyT','Qman','OPUS','neoHEVC','JacobSwaggedUp','NhaNc3','BugsFunny','EPSiLON','TENEIGHTY','Luvmichelle','WiKi','KOGi','K2','CPT','DeeJayAhmed','Plex','Cerberus','Hakata-Ramen','AMIABLE','HQE','A-S','EbP','SSA','AkihitoSubs','TrollHD','SARTRE','monkee','BiPOLAR','HBO','LOL','SAMPA','SbR','Ozlem','r00t','MKVHub','RSG','Langbard','REVOLT','HRiP','RZeroX','ExREN','CBM','Archie','D-Z0N3','CAFFEiNE','TvT','heponeko','MTBB','USURY','ColdYawn','battousai44','2DSamurai','ITSat','Codres','RyRo','HETeam','MEDiEVAL','MVGroup','UsifRenegade','Koten_Gars','B-A','Coalgirls','2HD','REGRET','Bearfish','Saizen','ZMNT','NiGHTMARE','W4F','HiDt','RTN','HaeRong','Orenji','HAGGiS','JK','Doki','SPRiNTER','CiNEFiLE','MTN','HxD','HR','DmonHiro','DHD','Prof','nikt0','PiRATEKiDS','Tenrai-Sensei','INFERNO','VietHD','OZC','uTs','CHD','uDF','TABULARiA','DIE','ETHiCS','GundamGuy','TOPAZ','BARTI','STEiNO','FraMeSToR','NanDesuKa','P-O','Anonymous','Positron','BluDragon','alfaHD','FQM','SH3LBY','JYK','YOGI','DragonFox','HoM','TayTO','SPASM','RB58','qcf','ION10','Burdock','Solar','TAoE','KHN','SiGMA','SiNDK8','Natty','ABM','GHOULS','OSiRiS','Moozzi2','Hurtom','Commie','LIGER','CBFM','NT','SacReD','Solonese','TBS','P0W4DVD','SADPANDA','AFG','FFF','BAJSKORV','hchcsen','entameSubs','Kametsu','Pir8','SallySubs','tlacatlc6','SMC','AMRAP','FFA','NOGPR','SMURF','YFN','iNSPiRiT','ColdFusion','playWEB','HQMUX','Bandi','iKA','LoTV','LRN','PublicHD','Slayer2K','l33t-raws','DiD','CHiPPER','SWTYBLZ','xRed','GCJM','Tenrai_Sensei','YesTV','ApachePrime','HighCode','Hon3y','STRiFE','SiN','Anime_Time','vRs','smplstc','PandaMoon','GERMini','Cakes','iT00NZ','ATV','BiA','Classics','D3FiL3R','SCY','PECULATE','Kappa','Alaboodi','SPARKS','FreetheFish','Yellow-Flash','Kira','anoXmous','MeGaTroN','KamiKaze','UTR','m00tv','Ainz','R-Raws','Aoi-WSRN-Licca','stuntkid','Arukoru','Mam','cmbt','GST','iNGOT','Raizel','MG28','BlackStrat','TBN','joseole99','SanKyuu','PTer','Holo','Silver_Zero_Subs','BIGDOC','Stay-Gold','WMV9','Paso77','YTS','GECKOS','usury','PHOBOS','HeVK','DDR','DVDRip','dvdrip','psyhd','TombDoc','Korach','xtream3x','SHUNPO','npuer','bipolar','hd4u','cinefile','geckos','leverage','Silence','MasterMind','HORiZON','SUPER','NEXT','Keyword','Pixel','KILLERS','EXTREME','Joy','MONOLITH','Ghost','Panda','LION','RUNNER','MOMENTUM','GHOSTS']

In [ ]:
##for x in encoders1:
  ##  print("'"+x+"'", end=',')

In [17]:
compiled_regex=re.compile('|'.join(encoders1))
compiled_regex

re.compile(r'30nama|YIFY|PaHe|RMTeam|PSA|RARBG|Ohys-Raws|HorribleSubs|Judas|AnimeRG|ASW|d3g|Tigole|AdiT|Erai-raws|Cleo|Phanteam|Ganool|MANiC|VXT|MkvCage|SS|EVO|LoveBug|DB|mSD|RCVR|NoGrouP|BTN|NoGrp|squalor|SAiNTS|MeGusta|NTb|CtrlHD|SubsPlease|Kirion|Goki|TVV|SA89|WELP|NODLABS|Vyndros|SNAKE|aAF|OSiTV|pseudo|DON|Sotark|Anime-Time|Yakubo|jlw|afm72|x0r|ViKi|EMBER|QOQ|Telly|AJP69|bonkai77|Reaktor|CMRG|HANDJOB|NTG|FGT|HoneyG|DEiMOS|GalaxyTV|Will1869|PHOCiS|zza|JiTB|iFT|MRN|MZABI|TEPES|ETRG|FFNDVD|Ravarage|SFM|ImE|ShAaNiG|Mana|GalaxyRG|PLAiD|WeTv|fov|CasStudio|Garshasp|AppleTor|KRaLiMaRKo|HDMaNiAcS|BTW|IPT|DarkDream|t3nzin|NyT|Qman|OPUS|neoHEVC|JacobSwaggedUp|NhaNc3|BugsFunny|EPSiLON|TENEIGHTY|Luvmichelle|WiKi|KOGi|K2|CPT|DeeJayAhmed|Plex|Cerberus|Hakata-Ramen|AMIABLE|HQE|A-S|EbP|SSA|AkihitoSubs|TrollHD|SARTRE|monkee|BiPOLAR|HBO|LOL|SAMPA|SbR|Ozlem|r00t|MKVHub|RSG|Langbard|REVOLT|HRiP|RZeroX|ExREN|CBM|Archie|D-Z0N3|CAFFEiNE|TvT|heponeko|MTBB|USURY|ColdYawn|battousai44|2DSamurai|ITSat|Codres|R

In [18]:
encoder_match=final.File_Path.apply(lambda x: re.findall(compiled_regex,x))
encoder_match

0        [Bandi]
1       [Tigole]
2       [DVDRip]
3        [afm72]
4        [Bandi]
         ...    
767        [VXT]
768     [30nama]
769     [dvdrip]
770           []
771    [Silence]
Name: File_Path, Length: 772, dtype: object

In [20]:
def convert_string(x):
    try:
        return x[-1]
    except :
        return np.nan

In [21]:
encoder_match=encoder_match.apply(convert_string)
encoder_match

0        Bandi
1       Tigole
2       DVDRip
3        afm72
4        Bandi
        ...   
767        VXT
768     30nama
769     dvdrip
770        NaN
771    Silence
Name: File_Path, Length: 772, dtype: object

In [22]:
df4=pd.DataFrame(encoder_match)

In [23]:
df4.rename(columns={'File_Path':'Encoder'},inplace=True)
df4

,Encoder
0,Bandi
1,Tigole
2,DVDRip
3,afm72
4,Bandi
...,...
767,VXT
768,30nama
769,dvdrip
770,NaN


In [24]:
final=pd.concat([final,df4],axis=1)
final

,Title,Folder_Path,File_Path,Encoder
0,Blue Is the Warmest Color (2013),G:\0 Director\Abdellatif Kechiche\Blue Is the Warmest Color (2013) Tigole 7.4 3000,Blue_Is_the_Warmest_Color_2013_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Bandi_30NAMA.mkv,Bandi
1,The Secret of the Grain (2007),G:\0 Director\Abdellatif Kechiche\The Secret Of The Grain (2007) Tigole 7.5 300,The_Secret_of_the_Grain_2007_CRITERION_COLLECTION_10bit_1080p_x265_BluRay_Tigole_30NAMA.mkv,Tigole
2,Black Venus (2010),G:\0 Director\Abdellatif Kechiche\Vénus noire (2010) dvd 7 100,Black_Venus_2010_CD1_DVDRip_RedBlade_30NAMA.avi,DVDRip
3,21 Grams (2003),G:\0 Director\Alejandro Inarritu\21 Grams (2003) Tigole 6.5,21 Grams (2003) (1080p BluRay x265 afm72).mkv,afm72
4,Amores Perros (2000),G:\0 Director\Alejandro Inarritu\Amores Perros (2000) Tigole 7.6,Amores Perros (2000) (1080p BluRay x265 Bandi).mkv,Bandi
...,...,...,...,...
767,Happy Hour (2015),G:\2015 - 2020\8 - 9\6 Happy Hour (2015) VXT 8.5 100,Happy Hour 2015 JAPANESE 720p BluRay H264 AAC-VXT.mkv,VXT
768,Dominion (2018),G:\2015 - 2020\8 - 9\7 Dominion (2018) YTS 8.6 0,Dominion_2018_1080p_WEB-DL_30nama_30NAMA.mkv,30nama
769,colliding dreams (2015),G:\2015 - 2020\8 - 9\8 Colliding Dreams (2015) dvd 8 0,colliding.dreams.2015.limited.dvdrip.x264.mkv,dvdrip
770,Cemetery of Splendor (2015),G:\2015 - 2020\cemetery.of.splendor.2015.limited.1080p.bluray.x264-depth,Cemetery.of.Splendor.2015.LIMITED.1080p.BluRay.x264-DEPTH.mkv,NaN


In [25]:
final.to_csv('hard10.csv',index=False)

In [ ]:
final=pd.read_csv('hard10.csv')
final

# Extract imdb id from API

In [35]:
moviesDB=imdb.IMDb()

In [36]:
movies=pd.read_csv('hard3.csv')

In [37]:
movies['ID']='tt'

In [46]:
#41 minutes
start_time = time.time()
for movie in movies['Title'][705:706]:
    s_movies=moviesDB.search_movie(movie)  
    try:
        asli_movie=s_movies[0]
    except:
            pass
    title=asli_movie['title']
    ID=asli_movie.getID()
    movies.loc[movies.Title==movie,'ID']='tt'+ID
time.time() - start_time

1.1067383289337158

In [47]:
movies

,Title,FolderPath,FilePath,Encoder,ID
0,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,tt0119008
1,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,tt0107818
2,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0159097
3,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,tt0120907
4,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,tt0119229
...,...,...,...,...,...
701,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,tt0119698
702,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0216625
703,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0151568
704,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,tt0151568


In [48]:
final=movies

In [49]:
final

,Title,FolderPath,FilePath,Encoder,ID
0,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,tt0119008
1,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,tt0107818
2,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0159097
3,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,tt0120907
4,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,tt0119229
...,...,...,...,...,...
701,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,tt0119698
702,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0216625
703,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0151568
704,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,tt0151568


In [50]:
final.to_csv('hard3.csv',index=False)

# Convert IMDB ID to TMDB

In [ ]:
movies=pd.read_csv('hard5.csv')

In [51]:
api_key='api_key=51f02a795f9a3321ba1ed1deb79c9443'

In [52]:
basic_url='https://api.themoviedb.org/3/movie/{}?{}'

In [53]:
movies

,Title,FolderPath,FilePath,Encoder,ID
0,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,tt0119008
1,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,tt0107818
2,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0159097
3,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,tt0120907
4,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,tt0119229
...,...,...,...,...,...
701,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,tt0119698
702,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0216625
703,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0151568
704,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,tt0151568


In [54]:
imdb_id=movies['ID']

In [55]:
json_list=[]

In [56]:
#30 minutes
for imdb in imdb_id:
    movies_url_imdb='https://api.themoviedb.org/3/find/'+imdb+'?'+api_key+'&external_source=imdb_id'
    try:
        r=requests.get(movies_url_imdb)
        if r.status_code !=200:
            continue
        else:
            data=r.json()
            json_list.append(data)
    except:
        pass
df=pd.DataFrame(json_list)

In [57]:
len(json_list)

706

In [ ]:
df.to_csv('moviedb.csv',index=False)

In [58]:
df.to_json('moviedb.json', orient='records')

In [59]:
with open('moviedb.json') as f:
    data=json.load(f)

In [60]:
bestdf=pd.json_normalize(data,'movie_results')

In [61]:
bestdf.to_csv('themoviedb.csv',index=False)

In [62]:
bestdf

,adult,backdrop_path,id,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count
0,False,/7aWpb0sfGteTomS9DP5NWDeD7DL.jpg,9366,Donnie Brasco,en,Donnie Brasco,An FBI undercover agent infilitrates the mob and finds himself identifying more with the mafia l...,/xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,movie,"[80, 18, 53]",21.262,1997-02-27,False,7.532,3621
1,False,/nWO4sGqH950N2fij8X0Pwumqe9k.jpg,9800,Philadelphia,en,Philadelphia,Two competing lawyers join forces to sue a prestigious law firm for AIDS discrimination. As thei...,/tFe5Yoo5zT495okA49bq1vPPkiV.jpg,movie,[18],19.276,1993-12-22,False,7.737,3540
2,False,/6b6UpsOXmjO8mJKYsGTG9HW49CN.jpg,1443,The Virgin Suicides,en,The Virgin Suicides,A group of male friends become obsessed with five mysterious sisters who are sheltered by their ...,/wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,movie,"[18, 10749]",22.197,1999-12-31,False,7.097,2526
3,False,/247OkG4qeUpy3WiKjilQyqNBwnY.jpg,1946,eXistenZ,en,eXistenZ,A game designer on the run from assassins must play her latest virtual reality creation with a m...,/kETKF0JhdTPn1knci8CAdYL0d79.jpg,movie,"[28, 53, 878, 27]",12.434,1999-04-14,False,6.839,1514
4,False,/gOIcpFDfCPsEV5wcmtbjrDZm2Av.jpg,9434,Grosse Pointe Blank,en,Grosse Pointe Blank,"Martin Blank is a hitman for hire. When he starts to develop a conscience, he botches a couple o...",/7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,movie,"[28, 35, 53, 10749]",16.910,1997-04-11,False,6.952,695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,False,/mpNd0rTVrp6vHJ9Je7wSa3zC8JS.jpg,128,Princess Mononoke,ja,もののけ姫,"Ashitaka, a prince of the disappearing Emishi people, is cursed by a demonized boar god and must...",/cMYCDADoLKLbB83g4WnJegaZimC.jpg,movie,"[12, 14, 16]",44.786,1997-07-12,False,8.350,6438
672,False,/bTUBuFR25xvTiFel7q6pq0ejacY.jpg,30970,Code Unknown,fr,Code inconnu,"A series of events unfold like a chain reaction, all stemming from a minor event that brings the...",/kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,movie,"[18, 10749, 10752]",9.016,2000-11-15,False,6.800,172
673,False,/lBk6VjaqbjU2T8wxiXW1zUh6M9r.jpg,46435,Topsy-Turvy,en,Topsy-Turvy,"After their production ""Princess Ida"" meets with less-than-stunning reviews, the relationship be...",/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,movie,"[35, 18, 10402, 36]",8.245,1999-12-15,False,6.700,115
674,False,/lBk6VjaqbjU2T8wxiXW1zUh6M9r.jpg,46435,Topsy-Turvy,en,Topsy-Turvy,"After their production ""Princess Ida"" meets with less-than-stunning reviews, the relationship be...",/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,movie,"[35, 18, 10402, 36]",8.245,1999-12-15,False,6.700,115


In [63]:
bestdf=bestdf.loc[:,['id','title','poster_path']]
bestdf

,id,title,poster_path
0,9366,Donnie Brasco,/xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg
1,9800,Philadelphia,/tFe5Yoo5zT495okA49bq1vPPkiV.jpg
2,1443,The Virgin Suicides,/wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg
3,1946,eXistenZ,/kETKF0JhdTPn1knci8CAdYL0d79.jpg
4,9434,Grosse Pointe Blank,/7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg
...,...,...,...
671,128,Princess Mononoke,/cMYCDADoLKLbB83g4WnJegaZimC.jpg
672,30970,Code Unknown,/kJJx2FNmiaep8WT21uMPY9qPzxL.jpg
673,46435,Topsy-Turvy,/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg
674,46435,Topsy-Turvy,/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg


In [64]:
bestdf.drop_duplicates(keep='first',inplace=True)
bestdf.reset_index(inplace=True,drop=True)

bestdf

,id,title,poster_path
0,9366,Donnie Brasco,/xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg
1,9800,Philadelphia,/tFe5Yoo5zT495okA49bq1vPPkiV.jpg
2,1443,The Virgin Suicides,/wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg
3,1946,eXistenZ,/kETKF0JhdTPn1knci8CAdYL0d79.jpg
4,9434,Grosse Pointe Blank,/7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg
...,...,...,...
637,11621,Porco Rosso,/byKAndF6KQSDpGxp1mTr23jPbYp.jpg
638,128,Princess Mononoke,/cMYCDADoLKLbB83g4WnJegaZimC.jpg
639,30970,Code Unknown,/kJJx2FNmiaep8WT21uMPY9qPzxL.jpg
640,46435,Topsy-Turvy,/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg


In [65]:
bestdf['download_poster']='https://image.tmdb.org/t/p/w600_and_h900_bestv2/'+bestdf.poster_path
bestdf

,id,title,poster_path,download_poster
0,9366,Donnie Brasco,/xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg
1,9800,Philadelphia,/tFe5Yoo5zT495okA49bq1vPPkiV.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg
2,1443,The Virgin Suicides,/wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg
3,1946,eXistenZ,/kETKF0JhdTPn1knci8CAdYL0d79.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg
4,9434,Grosse Pointe Blank,/7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg
...,...,...,...,...
637,11621,Porco Rosso,/byKAndF6KQSDpGxp1mTr23jPbYp.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//byKAndF6KQSDpGxp1mTr23jPbYp.jpg
638,128,Princess Mononoke,/cMYCDADoLKLbB83g4WnJegaZimC.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg
639,30970,Code Unknown,/kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg
640,46435,Topsy-Turvy,/vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg


In [66]:
bestdf.to_csv('Edited_best_df.csv',index=False)

### Get Director Name TMDB

In [67]:
api_key='api_key=51f02a795f9a3321ba1ed1deb79c9443'

In [68]:
tmdb_ID=bestdf['id']

In [69]:
tmdb_ID_part=tmdb_ID[0:642]

In [70]:
tmdb_ID_part=tmdb_ID_part.astype(str)

In [71]:
tmdb_full=[]

In [72]:
#20 minutes
for ID1 in tmdb_ID_part:
    movies_url_imdb='https://api.themoviedb.org/3/movie/'+ID1+'/credits?'+api_key
    try:
        r=requests.get(movies_url_imdb)
        if r.status_code !=200:
            continue
        else:
            full_data=r.json()
            tmdb_full.append(full_data)
    except:
        pass
df=pd.DataFrame(tmdb_full)

In [73]:
df.to_json('moviedb.json', orient='records')

In [74]:
with open('moviedb.json') as f:
    data=json.load(f)

In [75]:
Director=pd.json_normalize(data,'crew',['id'],meta_prefix='Movie')

In [138]:
Director

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,Movieid
0,False,2,949,Production,Louis DiGiaimo,Louis DiGiaimo,1.025,None,52fe44eec3a36847f80b278b,Production,Producer,9366
1,False,2,949,Production,Louis DiGiaimo,Louis DiGiaimo,1.025,None,5eab5b85c8a2d4001f49dfb2,Production,Casting,9366
2,False,2,45543,Writing,Paul Attanasio,Paul Attanasio,2.366,None,52fe44eec3a36847f80b2785,Writing,Screenplay,9366
3,False,1,4189,Costume & Make-Up,Aude Bronson-Howard,Aude Bronson-Howard,0.626,None,5eab5bbd99d5c300252d481d,Costume & Make-Up,Costume Design,9366
4,False,2,4190,Costume & Make-Up,David C. Robinson,David C. Robinson,3.842,None,5eab5bc81b157d001fc6fdf3,Costume & Make-Up,Costume Design,9366
...,...,...,...,...,...,...,...,...,...,...,...,...
25281,False,0,3391228,Costume & Make-Up,Carolina López,Carolina López,0.600,None,61e9fddc944a57009d4dafe3,Costume & Make-Up,Assistant Makeup Artist,140
25282,False,0,3391229,Costume & Make-Up,Caroline Turner,Caroline Turner,0.600,None,61e9fe9ca0b6b5001cfa08fd,Costume & Make-Up,Wigmaker,140
25283,False,0,3391231,Costume & Make-Up,Jessica Williams,Jessica Williams,0.600,None,61e9fec39c97bd0069334a56,Costume & Make-Up,Wigmaker,140
25284,False,0,3391232,Directing,Alicia Fdez. Alberdi,Alicia Fdez. Alberdi,0.600,None,61e9fef8fea0d70104dc240e,Directing,Assistant Director Trainee,140


In [77]:
Director_name=[]
temp1=Director.loc[Director.job=='Director']

In [157]:
temp1.drop_duplicates(subset='Movieid',keep='first',inplace=True)
temp2=temp1[['Movieid','name','profile_path']]
temp2.reset_index(drop=True,inplace=True)
temp2

C:\Users\zolfaghari\AppData\Local\Temp\ipykernel_21520\2908303048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1.drop_duplicates(subset='Movieid',keep='first',inplace=True)


,Movieid,name,profile_path
0,9366,Mike Newell,/zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg
1,9800,Jonathan Demme,/hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg
2,1443,Sofia Coppola,/dzHC2LxmarkBxWLhjp2DRa5oCev.jpg
3,1946,David Cronenberg,/kLHnBnUomcH96PdLRBnuZUzRoY1.jpg
4,9434,George Armitage,/rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg
...,...,...,...
637,11621,Hayao Miyazaki,/mG3cfxtA5jqDc7fpKgyzZMKoXDh.jpg
638,128,Hayao Miyazaki,/mG3cfxtA5jqDc7fpKgyzZMKoXDh.jpg
639,30970,Michael Haneke,/76Y4Li5lTbABoC8l9hR0khmmYuj.jpg
640,46435,Mike Leigh,/eQAc3gIjhAFTgHVQ3kfpbkUe97M.jpg


## download director posters

In [158]:
temp2.drop_duplicates(subset='name',keep='first',inplace=True)
temp2

C:\Users\zolfaghari\AppData\Local\Temp\ipykernel_21520\4181540906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2.drop_duplicates(subset='name',keep='first',inplace=True)


,Movieid,name,profile_path
0,9366,Mike Newell,/zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg
1,9800,Jonathan Demme,/hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg
2,1443,Sofia Coppola,/dzHC2LxmarkBxWLhjp2DRa5oCev.jpg
3,1946,David Cronenberg,/kLHnBnUomcH96PdLRBnuZUzRoY1.jpg
4,9434,George Armitage,/rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg
5,586,Barry Levinson,/gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg
6,226,Kimberly Peirce,/lRnwxog6xUZV26mxCVuMFM006CV.jpg
7,11005,Penny Marshall,/tENryuxNtctDMS26Tv5d8B7TclQ.jpg
8,687,Tim Robbins,/hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg
9,9559,Clint Eastwood,/dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg


In [159]:
temp2['download_director']='https://image.tmdb.org/t/p/w600_and_h900_bestv2/'+temp2.profile_path
temp2

C:\Users\zolfaghari\AppData\Local\Temp\ipykernel_21520\1132110161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['download_director']='https://image.tmdb.org/t/p/w600_and_h900_bestv2/'+temp2.profile_path


,Movieid,name,profile_path,download_director
0,9366,Mike Newell,/zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg
1,9800,Jonathan Demme,/hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg
2,1443,Sofia Coppola,/dzHC2LxmarkBxWLhjp2DRa5oCev.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dzHC2LxmarkBxWLhjp2DRa5oCev.jpg
3,1946,David Cronenberg,/kLHnBnUomcH96PdLRBnuZUzRoY1.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kLHnBnUomcH96PdLRBnuZUzRoY1.jpg
4,9434,George Armitage,/rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg
5,586,Barry Levinson,/gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg
6,226,Kimberly Peirce,/lRnwxog6xUZV26mxCVuMFM006CV.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//lRnwxog6xUZV26mxCVuMFM006CV.jpg
7,11005,Penny Marshall,/tENryuxNtctDMS26Tv5d8B7TclQ.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tENryuxNtctDMS26Tv5d8B7TclQ.jpg
8,687,Tim Robbins,/hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg
9,9559,Clint Eastwood,/dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg


In [165]:
temp2.dropna(subset='download_director',inplace=True)
temp2


C:\Users\zolfaghari\AppData\Local\Temp\ipykernel_21520\1021960347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2.dropna(subset='download_director',inplace=True)


,Movieid,name,profile_path,download_director
0,9366,Mike Newell,/zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg
1,9800,Jonathan Demme,/hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg
2,1443,Sofia Coppola,/dzHC2LxmarkBxWLhjp2DRa5oCev.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dzHC2LxmarkBxWLhjp2DRa5oCev.jpg
3,1946,David Cronenberg,/kLHnBnUomcH96PdLRBnuZUzRoY1.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kLHnBnUomcH96PdLRBnuZUzRoY1.jpg
4,9434,George Armitage,/rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg
5,586,Barry Levinson,/gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg
6,226,Kimberly Peirce,/lRnwxog6xUZV26mxCVuMFM006CV.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//lRnwxog6xUZV26mxCVuMFM006CV.jpg
7,11005,Penny Marshall,/tENryuxNtctDMS26Tv5d8B7TclQ.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tENryuxNtctDMS26Tv5d8B7TclQ.jpg
8,687,Tim Robbins,/hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg
9,9559,Clint Eastwood,/dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg


In [169]:
temp2=temp2.reset_index()
temp2.drop(columns=['index'],inplace=True)
temp2

,Movieid,name,profile_path,download_director
0,9366,Mike Newell,/zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//zeLnvLZtyJaBP1wWceRM7HWkm0E.jpg
1,9800,Jonathan Demme,/hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hm4wkbBOKaeLwMIk51hZ06OUkW3.jpg
2,1443,Sofia Coppola,/dzHC2LxmarkBxWLhjp2DRa5oCev.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dzHC2LxmarkBxWLhjp2DRa5oCev.jpg
3,1946,David Cronenberg,/kLHnBnUomcH96PdLRBnuZUzRoY1.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kLHnBnUomcH96PdLRBnuZUzRoY1.jpg
4,9434,George Armitage,/rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//rsVlCNOHKLUyo2xrA1jUlQmUUdx.jpg
5,586,Barry Levinson,/gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//gAVvEqbd9EBLiVBWArH7Poe0F1b.jpg
6,226,Kimberly Peirce,/lRnwxog6xUZV26mxCVuMFM006CV.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//lRnwxog6xUZV26mxCVuMFM006CV.jpg
7,11005,Penny Marshall,/tENryuxNtctDMS26Tv5d8B7TclQ.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tENryuxNtctDMS26Tv5d8B7TclQ.jpg
8,687,Tim Robbins,/hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//hsCu1JUzQQ4pl7uFxAVFLOs9yHh.jpg
9,9559,Clint Eastwood,/dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg,https://image.tmdb.org/t/p/w600_and_h900_bestv2//dU35NnjZ4aGw5abIJe3WXVf3Eey.jpg


In [170]:
iterator=temp2.download_director[0:285]

In [171]:
start_time = time.time()
for idx,i in enumerate(iterator):

  
        img_data = requests.get(i).content

        with open('{}.jpg'.format(temp2.name[idx]), 'wb') as handler:
            handler.write(img_data)
time.time()-start_time

223.74901628494263

### making folder for directors

In [173]:
Director_name = []
rootdir = 'C:\\Users\\zolfaghari\\Sample3'
for dirpath, subdirs, files in os.walk(rootdir):
    for x in files:
        if x.endswith(".ico"):
            Director_name.append(os.path.join(dirpath, x))
            
Director_name

['C:\\Users\\zolfaghari\\Sample2\\Abbas Kiarostami.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Aki Kaurismäki.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Aleksandr Sokurov.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Ali Hatami.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Ang Lee.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Bahman Ghobadi.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Bahram Beyzai.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Béla Tarr.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Christopher Guest.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Claire Denis.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Clint Eastwood.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Danny Boyle.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Dariush Mehrjui.ico',
 'C:\\Users\\zolfaghari\\Sample2\\David Cronenberg.ico',
 'C:\\Users\\zolfaghari\\Sample2\\David Lynch.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Edward Yang.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Giuseppe Tornatore.ico',
 'C:\\Users\\zolfaghari\\Sample2\\Gus Van Sant.ico',
 'C:\\Users\\zolfaghari\\

In [176]:
splited_path=[]
file_name=[]
for path in Director_name:
    spliting=os.path.split(path)
    splited_path.append(spliting[0])
    file_name.append(spliting[1])

In [182]:
new_file_name=[]

for x in file_name:
    new_file_name.append(x.replace('.ico',''))
    
new_file_name

['Abbas Kiarostami',
 'Aki Kaurismäki',
 'Aleksandr Sokurov',
 'Ali Hatami',
 'Ang Lee',
 'Bahman Ghobadi',
 'Bahram Beyzai',
 'Béla Tarr',
 'Christopher Guest',
 'Claire Denis',
 'Clint Eastwood',
 'Danny Boyle',
 'Dariush Mehrjui',
 'David Cronenberg',
 'David Lynch',
 'Edward Yang',
 'Giuseppe Tornatore',
 'Gus Van Sant',
 'Hayao Miyazaki',
 'Hou Hsiao-hsien',
 'Jean-Pierre Jeunet',
 'Jim Jarmusch',
 'Joel Coen',
 'John Woo',
 'Ken Loach',
 'Kim Ki-duk',
 'Lars von Trier',
 'Leos Carax',
 'Lukas Moodysson',
 'Majid Majidi',
 'Martin Scorsese',
 'Michael Haneke',
 'Michael Mann',
 'Mike Leigh',
 'Mohammad Rasoulof',
 'Mohsen Makhmalbaf',
 'Nanni Moretti',
 'Paolo Taviani',
 'Paul Schrader',
 'Pedro Almodóvar',
 'Peter Jackson',
 'Raúl Ruiz',
 'Ridley Scott',
 'Shunji Iwai',
 'Steven Spielberg',
 'Tony Gatlif',
 'Tsai Ming-liang',
 'Whit Stillman',
 'Wong Kar-wai',
 'Zeki Demirkubuz',
 'Zhang Yimou']

In [185]:
src=r'C:\Users\zolfaghari\Sample1'
for x in new_file_name:
    try:
        dst='C:\\Users\\zolfaghari\\Sample3\\'+x
        shutil.copytree(src,dst)
    except:
        continue

In [186]:
for x,y in list(zip(new_file_name,new_file_name)):

    try:
        src='C:\\Users\\zolfaghari\\Sample3\\'+y+'.ico'
        dst='C:\\Users\\zolfaghari\\Sample3\\'+x
        shutil.copy(src,dst)
    except:
        continue
    os.chdir('C:\\Users\\zolfaghari\\Sample3\\'+x)
    os.rename('C:\\Users\\zolfaghari\\Sample3\\'+y+'.ico','1.ico')
    
os.chdir('C:\\Users\\zolfaghari')

---------------------------------

In [79]:
tmdb_final=bestdf.merge(temp2,how='left',left_on='id',right_on='Movieid')
tmdb_final.drop(columns=['poster_path','Movieid'],inplace=True)
tmdb_final.rename(columns={'name':'Director'},inplace=True)
tmdb_final

,id,title,download_poster,Director
0,9366,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell
1,9800,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme
2,1443,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola
3,1946,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg
4,9434,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage
...,...,...,...,...
637,11621,Porco Rosso,https://image.tmdb.org/t/p/w600_and_h900_bestv2//byKAndF6KQSDpGxp1mTr23jPbYp.jpg,Hayao Miyazaki
638,128,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki
639,30970,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke
640,46435,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh


In [80]:
tmdb_final.to_csv('tmdb_final.csv',index=False)

### Get full data TMDB

In [81]:
tmdb_ID=bestdf['id']

In [82]:
tmdb_ID_part=tmdb_ID[0:706]

In [83]:
tmdb_ID_part=tmdb_ID_part.astype(str)

In [84]:
tmdb_full=[]

In [85]:

for ID1 in tmdb_ID_part:
    movies_url_imdb='https://api.themoviedb.org/3/movie/'+ID1+'?'+api_key
    try:
        r=requests.get(movies_url_imdb)
        if r.status_code !=200:
            continue
        else:
            full_data=r.json()
            tmdb_full.append(full_data)
    except:
        pass
df=pd.DataFrame(tmdb_full)

In [86]:
df1=df[['id','imdb_id']]
df1

,id,imdb_id
0,9366,tt0119008
1,9800,tt0107818
2,1443,tt0159097
3,1946,tt0120907
4,9434,tt0119229
...,...,...
637,11621,tt0104652
638,128,tt0119698
639,30970,tt0216625
640,46435,tt0151568


In [87]:
tmdb_final_imdb=tmdb_final.merge(df1,how='left',left_on='id',right_on='id')
tmdb_final_imdb

,id,title,download_poster,Director,imdb_id
0,9366,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008
1,9800,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818
2,1443,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097
3,1946,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907
4,9434,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229
...,...,...,...,...,...
637,11621,Porco Rosso,https://image.tmdb.org/t/p/w600_and_h900_bestv2//byKAndF6KQSDpGxp1mTr23jPbYp.jpg,Hayao Miyazaki,tt0104652
638,128,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki,tt0119698
639,30970,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke,tt0216625
640,46435,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568


In [88]:
final=pd.read_csv('hard3.csv')
final


,Title,FolderPath,FilePath,Encoder,ID
0,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,tt0119008
1,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,tt0107818
2,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0159097
3,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,tt0120907
4,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,tt0119229
...,...,...,...,...,...
701,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,tt0119698
702,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0216625
703,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,tt0151568
704,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,tt0151568


In [89]:
Final_full=tmdb_final_imdb.merge(final,how='right',left_on='imdb_id',right_on='ID')
Final_full.drop(columns=['ID'],inplace=True)
Final_full

,id,title,download_poster,Director,imdb_id,Title,FolderPath,FilePath,Encoder
0,9366,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72
1,9800,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72
2,1443,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole
3,1946,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole
4,9434,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t
...,...,...,...,...,...,...,...,...,...
701,128,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki,tt0119698,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole
702,30970,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke,tt0216625,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole
703,46435,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole
704,46435,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN


In [90]:
Final_full.to_csv('Final_full_hard3.csv',index=False)

# Scraping Criticker

In [91]:
df=pd.read_csv('Final_full_hard3.csv')
df.rename(columns={'imdb_id':'tconst'},inplace=True)
df

,id,title,download_poster,Director,tconst,Title,FolderPath,FilePath,Encoder
0,9366.0,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72
1,9800.0,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72
2,1443.0,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole
3,1946.0,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole
4,9434.0,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t
...,...,...,...,...,...,...,...,...,...
701,128.0,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki,tt0119698,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole
702,30970.0,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke,tt0216625,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole
703,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole
704,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN


In [92]:
len(df['tconst'])

706

In [99]:
iterator=df['tconst'][500:706]
iterator

500     tt0259072
501     tt0079997
502     tt0087855
503     tt0415127
504     tt1316540
505     tt0249241
506     tt2821088
507     tt0094868
508     tt0295743
509           NaN
510     tt0100536
511     tt0422491
512     tt0204700
513     tt0116282
514     tt1403865
515           NaN
516     tt0117951
517     tt0065376
518     tt0099729
519     tt0116851
520     tt0064356
521     tt0187809
522     tt0099492
523     tt0415607
524     tt0074486
525     tt0460829
526     tt0116922
527     tt0166896
528     tt0080678
529     tt0100935
530     tt0244316
531     tt1081935
532     tt0095765
533     tt1924396
534     tt1924396
535     tt0494271
536     tt0089537
537     tt0092067
538     tt0096283
539     tt0087544
540     tt0876563
541     tt0245429
542     tt0079833
543     tt2013293
544     tt0412596
545     tt0826711
546     tt0283283
547     tt0459666
548     tt0211915
549     tt0379217
550     tt0112817
551     tt0102536
552     tt0109688
553     tt0068569
554     tt0099768
555     tt

In [94]:
from time import sleep

In [100]:
# 1.5 Hours
start_time = time.time()
from tqdm.notebook import tqdm
    
with requests.session() as s:
    loginurl = 'https://www.criticker.com/authenticate.php'
    secureurl = 'https://www.criticker.com/'

    payload = {
    'si_username' : 'mamali_zoli',
    'si_password' : '728728'
    }
    try:
        s.post(loginurl, data=payload)
        r = s.get(secureurl)
    except:
        pass
    soup = BeautifulSoup(r.text, 'html.parser')

    time.sleep(0.4)

    for g,i in zip(iterator,tqdm(range(len(iterator)))):
        try:
            if math.isnan(g):
                continue
        except:
            pass
        

        address='https://www.criticker.com/?search='+g+'&type=films'

        try:
            r1 = s.get(address)
        except:
            pass

        soup_link_finder = BeautifulSoup(r1.text, 'html.parser')
        main_link = soup_link_finder.find_all('div', {'class': 'titlerow_name'})

        try:
            if len(main_link)!=0:
                criticker_link=main_link[0].a['href']
            else:
                criticker_link='NA'
                df.loc[df.tconst==g,'Criticker_Score']='Na'
                df.loc[df.tconst==g,'Criticker_Count']='Na'
                df.loc[df.tconst==g,'Criticker_link']='Na'
                df.loc[df.tconst==g,'My_Score']='Na'
                continue
                
        except:
            pass
        movie_name=criticker_link.replace('https://www.criticker.com/film/','')

        time.sleep(0.4)
        try:
            r2 = s.get(criticker_link)
        except:
            pass

        soup_main = BeautifulSoup(r2.text, 'html.parser')

        time.sleep(0.4)
        psi_address='https://www.criticker.com/psi/'+movie_name
        try:
            r3 = s.get(psi_address)
        except:
            pass
        soup_my_psi = BeautifulSoup(r3.text, 'html.parser')

        Criticker_score = soup_main.find_all('span', {'itemprop': 'ratingValue'})
        name = soup_main.find_all('span', {'itemprop': 'name'})
        count = soup_main.find_all('span', {'itemprop': 'reviewCount'})
        My_score = soup_my_psi.find_all('div', {'class': 'pe_footerdiv'})
        try:
            df.loc[df.tconst==g,'Criticker_Score']=Criticker_score[0].text
        except:
            df.loc[df.tconst==g,'Criticker_Score']='Na'
        try:    
            df.loc[df.tconst==g,'My_Score']=My_score[0].text
        except:
            df.loc[df.tconst==g,'My_Score']='Na'
        try:
            df.loc[df.tconst==g,'Criticker_Count']=count[0].text
        except:
            df.loc[df.tconst==g,'Criticker_Count']='Na'
        try:
            df.loc[df.tconst==g,'Criticker_link']=criticker_link
        except:
            df.loc[df.tconst==g,'Criticker_link']='Na'
            

df.to_csv('temp_movie_save.csv')  
time.time() - start_time   


  0%|          | 0/206 [00:00<?, ?it/s]

474.47639060020447

In [96]:
df[0:200]

,id,title,download_poster,Director,tconst,Title,FolderPath,FilePath,Encoder,Criticker_Score,My_Score,Criticker_Count,Criticker_link
0,9366.0,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,6.71,5.3,5696,https://www.criticker.com/film/Donnie-Brasco/
1,9800.0,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,6.53,5.5,5515,https://www.criticker.com/film/Philadelphia/
2,1443.0,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.07,6.3,5035,https://www.criticker.com/film/The-Virgin-Suicides/
3,1946.0,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,5.76,6.7,4412,https://www.criticker.com/film/eXistenZ/
4,9434.0,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,6.38,5.3,3508,https://www.criticker.com/film/Grosse-Pointe-Blank/
5,586.0,Wag the Dog,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kAwOj7t8ZeaniTFtLWVSKO79JxX.jpg,Barry Levinson,tt0120885,Wag the Dog (1997),F:\1990 Decade\6 - 6.5\17 Wag the Dog (1997) afm72 6.1 3400,Wag the Dog (1997) (1080p WEB-DL x265 afm72).mkv,afm72,5.94,6.2,3404,https://www.criticker.com/film/Wag-the-Dog/
6,226.0,Boys Don't Cry,https://image.tmdb.org/t/p/w600_and_h900_bestv2//lPg3kFAedCIHLSE8YxTBb97wAMM.jpg,Kimberly Peirce,tt0171804,Boys Don't Cry (1999),F:\1990 Decade\6 - 6.5\18 Boys Dont Cry (1999) Silence 6.2 3100,Boys Don't Cry (1999) (1080p BluRay x265 Silence).mkv,Silence,6.55,5.8,3048,https://www.criticker.com/film/Boys-Dont-Cry/
7,11005.0,Awakenings,https://image.tmdb.org/t/p/w600_and_h900_bestv2//9gztZXuHLG6AJ0fgqGd7Q43cWRI.jpg,Penny Marshall,tt0099077,Awakenings (1990),F:\1990 Decade\6 - 6.5\19 Awakenings (1990) Tigole 6.4 2800,Awakenings (1990) (1080p BluRay x265 10bit Tigole).mkv,Tigole,6.61,6.1,2813,https://www.criticker.com/film/Awakenings/
8,687.0,Dead Man Walking,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wQmmJi5ypfHH2boXrQBmsep7qb2.jpg,Tim Robbins,tt0112818,Dead Man Walking (1995),F:\1990 Decade\6 - 6.5\21 Dead Man Walking (1995) Silence 6.1 2700,Dead Man Walking (1995) (1080p BluRay x265 Silence).mkv,Silence,6.53,5.6,2728,https://www.criticker.com/film/Dead-Man-Walking/
9,9559.0,A Perfect World,https://image.tmdb.org/t/p/w600_and_h900_bestv2//1II9kMjOuho3pnEvxkpx7e3ozOO.jpg,Clint Eastwood,tt0107808,A Perfect World (1993),F:\1990 Decade\6 - 6.5\24 A Perfect World (1993) r00t 6.1 1500,A Perfect World (1993) (1080p BluRay x265 r00t).mkv,r00t,6.67,5.4,1531,https://www.criticker.com/film/A-Perfect-World/


In [101]:
df=pd.read_csv('temp_movie_save.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df

,id,title,download_poster,Director,tconst,Title,FolderPath,FilePath,Encoder,Criticker_Score,My_Score,Criticker_Count,Criticker_link
0,9366.0,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,6.71,5.3,5696,https://www.criticker.com/film/Donnie-Brasco/
1,9800.0,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,6.53,5.5,5515,https://www.criticker.com/film/Philadelphia/
2,1443.0,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.07,6.3,5035,https://www.criticker.com/film/The-Virgin-Suicides/
3,1946.0,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,5.76,6.7,4412,https://www.criticker.com/film/eXistenZ/
4,9434.0,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,6.38,5.3,3508,https://www.criticker.com/film/Grosse-Pointe-Blank/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,128.0,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki,tt0119698,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,7.83,7.9,8859,https://www.criticker.com/film/Princess-Mononoke/
702,30970.0,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke,tt0216625,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.68,8.0,793,https://www.criticker.com/film/Code-Unknown/
703,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.78,6.6,455,https://www.criticker.com/film/Topsy-Turvy/
704,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,6.78,6.6,455,https://www.criticker.com/film/Topsy-Turvy/


#### Create mean score criticker

In [102]:
df.My_Score.replace('Na',np.nan,inplace=True)
df.My_Score.replace('NA',np.nan,inplace=True)
df.Criticker_Score.replace('Na',np.nan,inplace=True)
df.Criticker_Score.replace('NA',np.nan,inplace=True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       706 non-null    object
 1   FolderPath  706 non-null    object
 2   FilePath    706 non-null    object
 3   Encoder     645 non-null    object
 4   ID          706 non-null    object
dtypes: object(5)
memory usage: 27.7+ KB


In [103]:
df.My_Score=df.My_Score.astype('float')
df.Criticker_Score=df.Criticker_Score.astype('float')

In [104]:
df.loc[df.My_Score.isnull(),'Mean_Criticker']=np.nan
df.loc[df.Criticker_Score.isnull(),'Mean_Criticker']=np.nan

In [105]:
df.loc[df.My_Score.notna(),'Mean_Criticker']=(df.My_Score + df.Criticker_Score)/2

In [106]:
df

,id,title,download_poster,Director,tconst,Title,FolderPath,FilePath,Encoder,Criticker_Score,My_Score,Criticker_Count,Criticker_link,Mean_Criticker
0,9366.0,Donnie Brasco,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,Mike Newell,tt0119008,Donnie Brasco (1997),F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,afm72,6.71,5.3,5696,https://www.criticker.com/film/Donnie-Brasco/,6.005
1,9800.0,Philadelphia,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,Jonathan Demme,tt0107818,Philadelphia (1993),F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,afm72,6.53,5.5,5515,https://www.criticker.com/film/Philadelphia/,6.015
2,1443.0,The Virgin Suicides,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,Sofia Coppola,tt0159097,The Virgin Suicides (1999),F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.07,6.3,5035,https://www.criticker.com/film/The-Virgin-Suicides/,6.185
3,1946.0,eXistenZ,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,David Cronenberg,tt0120907,eXistenZ (1999),F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,Tigole,5.76,6.7,4412,https://www.criticker.com/film/eXistenZ/,6.230
4,9434.0,Grosse Pointe Blank,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,George Armitage,tt0119229,Grosse Pointe Blank (1997),F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,r00t,6.38,5.3,3508,https://www.criticker.com/film/Grosse-Pointe-Blank/,5.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,128.0,Princess Mononoke,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,Hayao Miyazaki,tt0119698,Princess Mononoke (1997),F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,Tigole,7.83,7.9,8859,https://www.criticker.com/film/Princess-Mononoke/,7.865
702,30970.0,Code Unknown,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,Michael Haneke,tt0216625,Code Unknown (2000),F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.68,8.0,793,https://www.criticker.com/film/Code-Unknown/,7.340
703,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,Topsy-Turvy (1999),F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,Tigole,6.78,6.6,455,https://www.criticker.com/film/Topsy-Turvy/,6.690
704,46435.0,Topsy-Turvy,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,Mike Leigh,tt0151568,"Ming-liang Tsai, (2005)",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",NaN,6.78,6.6,455,https://www.criticker.com/film/Topsy-Turvy/,6.690


In [107]:
df=df.loc[:,['Title','Director','Mean_Criticker','Criticker_Count','Encoder','FilePath','FolderPath','tconst','Criticker_link','title','id','download_poster','My_Score','Criticker_Score']]
df

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score
0,Donnie Brasco (1997),Mike Newell,6.005,5696,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71
1,Philadelphia (1993),Jonathan Demme,6.015,5515,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53
2,The Virgin Suicides (1999),Sofia Coppola,6.185,5035,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07
3,eXistenZ (1999),David Cronenberg,6.230,4412,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76
4,Grosse Pointe Blank (1997),George Armitage,5.840,3508,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.865,8859,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83
702,Code Unknown (2000),Michael Haneke,7.340,793,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68
703,Topsy-Turvy (1999),Mike Leigh,6.690,455,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.690,455,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78


In [108]:
df=df.round({'Mean_Criticker':1})


In [109]:
df.Criticker_Count.replace('Na',0,inplace=True)
df.Criticker_Count.replace('NA',0,inplace=True)
df.Criticker_Count=df.Criticker_Count.fillna(0)
df.Criticker_Count=df.Criticker_Count.astype('int')

In [110]:
df['Criticker_Count']=df['Criticker_Count'].apply(lambda x : round(x /250,0) * 250 if x!=np.nan else x)

In [111]:
df.Criticker_Count=df.Criticker_Count.astype('int')

In [112]:
cols = ['Title','Encoder', 'Mean_Criticker','Criticker_Count', 'Director']
df['Combination'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh"


In [113]:
df.to_csv('Final_full_hard3.csv',index=False)

# Scraping imdb

# Download posters with name and send to photoshop and x2ico

In [ ]:
os.chdir("zolfaghari")

In [ ]:
os.getcwd()
os.chdir("..")
os.getcwd()

In [114]:
Final_full=pd.read_csv('Final_full_hard3.csv',index_col=False)
Final_full

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh"


In [115]:
Final_full

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh"


In [121]:
iterator=Final_full.download_poster[0:706]

In [117]:
iterator

0      https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg
1      https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg
2      https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg
3      https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg
4      https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg
5      https://image.tmdb.org/t/p/w600_and_h900_bestv2//kAwOj7t8ZeaniTFtLWVSKO79JxX.jpg
6      https://image.tmdb.org/t/p/w600_and_h900_bestv2//lPg3kFAedCIHLSE8YxTBb97wAMM.jpg
7      https://image.tmdb.org/t/p/w600_and_h900_bestv2//9gztZXuHLG6AJ0fgqGd7Q43cWRI.jpg
8      https://image.tmdb.org/t/p/w600_and_h900_bestv2//wQmmJi5ypfHH2boXrQBmsep7qb2.jpg
9      https://image.tmdb.org/t/p/w600_and_h900_bestv2//1II9kMjOuho3pnEvxkpx7e3ozOO.jpg
10     https://image.tmdb.org/t/p/w600_and_h900_bestv2//cuCkhiuhPhb0wo05pgO209l3sk9.jpg
11     https://image.tmdb.org/t/

In [ ]:
# if intrupted we should change old idx+new in Final_full.Title[idx]

In [122]:
start_time = time.time()
for idx,i in enumerate(iterator):
    try:
        if math.isnan(i):
            continue
    except:
  
        img_data = requests.get(i).content

        with open('{}.jpg'.format(Final_full.Title[idx]), 'wb') as handler:
            handler.write(img_data)
time.time()-start_time

499.76968908309937

### make two folder for posters and icon and open photoshop File>automate>batch and open actions and put psd on desktop

In [ ]:
# make sure save log in photoshop issue

#### download link for psd and action file

In [ ]:
#https://mega.nz/file/MwhVnCpA#j2ErZPQstbHDakobHjTDtVy3APt4uUqM8nfy5HQuziw
#https://mega.nz/file/QkRFRbZR#2BmPsUsu5JcYvoljEXMCQjt6Pkqa0ZQ0GE_pOYWzjqM

#### after making poster put them on x2ico and preserve in one folder

# Copy sample Folder with Desktop.ini and rename folders

In [ ]:
Final_full=pd.read_csv('Final_full_hard3.csv',index_col=False)
Final_full

In [ ]:
#Final_full.loc[Final_full.Title=='The Set-Up (1949)']

In [129]:
names=Final_full.Combination[0:706]
names

0                 Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1               Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2        The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3                eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4        Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
                                  ...                            
701       Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702             Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703                  Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704                Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh
705    La Mala Educacion (2004) HighCode 7.0 2250 Pedro Almodóvar
Name: Combination, Length: 706, dtype: object

In [ ]:
#names.drop_duplicates(keep='first',inplace=True)


In [ ]:
#os.listdir(os.getcwd())
#os.chdir('Sample')
#os.getcwd()

#### making folders with full pattern names and desktop.ini

In [130]:
src=r'C:\Users\zolfaghari\Sample1'
for x in names:
    try:
        dst='C:\\Users\\zolfaghari\\Sample3\\'+x
        shutil.copytree(src,dst)
    except:
        continue
    #### after making poster put them on x2ico and preserve in one folder

#### copy poster icon to the folder

#### after making poster put them on x2ico and preserve in one sample folder

In [131]:
icon_name=Final_full.Title[0:706]

In [ ]:
#icon_name.dropna(inplace=True)

In [ ]:
len(icon_name)

In [133]:
for x,y in list(zip(names,icon_name)):

    try:
        src='C:\\Users\\zolfaghari\\Sample3\\'+y+'.ico'
        dst='C:\\Users\\zolfaghari\\Sample3\\'+x
        shutil.copy(src,dst)
    except:
        continue
    os.chdir('C:\\Users\\zolfaghari\\Sample3\\'+x)
    os.rename('C:\\Users\\zolfaghari\\Sample3\\'+y+'.ico','1.ico')
    
os.chdir('C:\\Users\\zolfaghari')

In [ ]:
list(zip(names,icon_name))

# top 100 of hard

In [134]:
df=pd.read_csv('Final_full_hard3.csv')
df

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh"


In [124]:
i=df.Criticker_Count.nlargest(100).index

In [125]:
new=df.iloc[i,:]
new

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
241,The Matrix (1999),Lilly Wachowski,6.7,31000,D-Z0N3,The.Matrix.1999.1080p.BluRay.DTS.x264-D-Z0N3.mkv,F:\1990 Decade\6.5 - 7\1 The Matrix (1999) US Z0N3 6.5 31000,tt0133093,https://www.criticker.com/film/The-Matrix/,The Matrix,603.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg,5.6,7.73,The Matrix (1999) D-Z0N3 6.7 31000 Lilly Wachowski
352,Forrest Gump (1994),Robert Zemeckis,5.8,27000,YIFY,Forrest_Gump_1994_10bit_2160p_x265_WEB-DL_5.1CH_YIFY_30NAMA.mkv,F:\1990 Decade\6 - 6.5\1 Forrest Gump (1994) 4K 6 27000,tt0109830,https://www.criticker.com/film/Forrest-Gump/,Forrest Gump,13.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//saHP97rTPS5eLmrLQEcANmKrsFl.jpg,4.3,7.24,Forrest Gump (1994) YIFY 5.8 27000 Robert Zemeckis
38,The Shawshank Redemption (1994),Frank Darabont,7.2,25000,Tigole,The Shawshank Redemption (1994) (2160p BluRay x265 10bit HDR Tigole).mkv,F:\1990 Decade\7 - 7.5\1 The Shawshank Redemption (1994) 4K Tigole 7.3 24700,tt0111161,https://www.criticker.com/film/The-Shawshank-Redemption/,The Shawshank Redemption,278.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,5.9,8.40,The Shawshank Redemption (1994) Tigole 7.2 25000 Frank Darabont
307,Eternal Sunshine of the Spotless Mind (2004),Michel Gondry,8.2,23750,SARTRE,Eternal.Sunshine.of.the.Spotless.Mind.2004.1080p.BluRay.x265.HEVC.EAC3-SARTRE.mkv,F:\2000-2005\8 - 9\Eternal Sunshine of the Spotless Mind (2004) Sartre 8.3 23700,tt0338013,https://www.criticker.com/film/Eternal-Sunshine-of-the-Spotless/,Eternal Sunshine of the Spotless Mind,38.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//5MwkWH9tYHv3mV9OdYTMR5qreIz.jpg,8.8,7.57,Eternal Sunshine of the Spotless Mind (2004) SARTRE 8.2 23750 Michel Gondry
211,Gladiator (2000),Ridley Scott,5.8,22000,afm72,Gladiator (2000) Extended (1080p BluRay x265 HDR afm72).mkv,F:\Director\Ridley Scott\[Zipped]Gladiator (2000) Extended (1080p BluRay x265 HEVC 10bit HDR AAC...,tt0172495,https://www.criticker.com/film/Gladiator-2000/,Gladiator,98.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//ty8TGRuvJLPUmAR1H1nRIsgwvim.jpg,4.8,6.89,Gladiator (2000) afm72 5.8 22000 Ridley Scott
688,Titanic (1997),James Cameron,4.3,21500,Joy,Titanic_1997_OPEN_MATTE_10bit_2160p_x265_BluRay_UHD_Joy_30NAMA.mkv,F:\hard\Titanic (1997) 4K 4.5,tt0120338,https://www.criticker.com/film/Titanic-1997/,Titanic,597.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,3.4,5.21,Titanic (1997) Joy 4.3 21500 James Cameron
407,The Silence of the Lambs (1991),Jonathan Demme,7.4,21250,YIFY,The_Silence_of_the_Lambs_1991_10bit_2160p_x265_WEB-DL_5.1CH_YIFY_30NAMA.mkv,F:\1990 Decade\7.5 - 8\1 The Silence of Lambs (1991) 4K YTS 7.5 21000,tt0102926,https://www.criticker.com/film/The-Silence-of-the-Lambs/,The Silence of the Lambs,274.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//rplLJ2hPcOQmkFhTqUte0MkEaO2.jpg,7.0,7.85,The Silence of the Lambs (1991) YIFY 7.4 21250 Jonathan Demme
135,The Big Lebowski (1998),Joel Coen,7.1,20250,afm72,The Big Lebowski (1998) (1080p BluRay x265 HDR afm72).mkv,F:\Director\Coen Brothers\The Big Lebowski ( 1998 ) Tigole 7.3,tt0118715,https://www.criticker.com/film/The-Big-Lebowski/,The Big Lebowski,115.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//5DpmtMBXXNDujIuSlKW3WLKuqEd.jpg,6.6,7.54,The Big Lebowski (1998) afm72 7.1 20250 Joel Coen
223,Donnie Darko (2001),Richard Kelly,6.1,20250,Silence,Donnie Darko (2001) (1080p BluRay x265 Silence).mkv,F:\hard\Donnie Darko (2001) Silence 5.9 20200,tt0246578,https://www.criticker.com/film/Donnie-Darko/,Donnie Darko,141.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//fhQoQfejY1hUcwyuLgpBrYs6uFt.jpg,5.5,6.79,Donnie Darko (2001) Silence 6.1 20250 Richard Kelly
671,Catch Me If You Can (2002),Steven Spielberg,5.8,20000,Tigole,Catch_Me_If_You_Can_20

In [126]:
new.to_csv('top100_hard3.csv',index=False)

In [128]:
df['Hard']='3'
df.to_csv('Final_full_hard31.csv')
df

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination,Hard
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell,3
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme,3
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola,3
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg,3
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki,3
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke,3
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh,3
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh",3


----------------------

In [136]:
df=pd.read_csv('Final_full_hard3.csv')
df

,Title,Director,Mean_Criticker,Criticker_Count,Encoder,FilePath,FolderPath,tconst,Criticker_link,title,id,download_poster,My_Score,Criticker_Score,Combination
0,Donnie Brasco (1997),Mike Newell,6.0,5750,afm72,Donnie Brasco (1997) Extended (1080p BluRay x265 afm72).mkv,F:\1990 Decade\6 - 6.5\10 Donnie Brasco (1997) afm72 6 5600,tt0119008,https://www.criticker.com/film/Donnie-Brasco/,Donnie Brasco,9366.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//xtKLvpOfARi1XVm8u2FTdhY5Piq.jpg,5.3,6.71,Donnie Brasco (1997) afm72 6.0 5750 Mike Newell
1,Philadelphia (1993),Jonathan Demme,6.0,5500,afm72,Philadelphia (1993) (1080p BluRay x265 HDR afm72).mkv,F:\1990 Decade\6 - 6.5\11 Philadelphia (1993) afm72 6 5500,tt0107818,https://www.criticker.com/film/Philadelphia/,Philadelphia,9800.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//tFe5Yoo5zT495okA49bq1vPPkiV.jpg,5.5,6.53,Philadelphia (1993) afm72 6.0 5500 Jonathan Demme
2,The Virgin Suicides (1999),Sofia Coppola,6.2,5000,Tigole,The Virgin Suicides (1999) Criterion (1080p x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\12 The Virgin Suicides (1999) Tigole 6.2 5000,tt0159097,https://www.criticker.com/film/The-Virgin-Suicides/,The Virgin Suicides,1443.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//wDrTeDUuWf7SRQnkVea2ZpOcwnt.jpg,6.3,6.07,The Virgin Suicides (1999) Tigole 6.2 5000 Sofia Coppola
3,eXistenZ (1999),David Cronenberg,6.2,4500,Tigole,eXistenZ (1999) (1080p BluRay x265 10bit Tigole).mkv,F:\1990 Decade\6 - 6.5\13 eXistenZ (1999) Tigole 6.2 4400,tt0120907,https://www.criticker.com/film/eXistenZ/,eXistenZ,1946.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kETKF0JhdTPn1knci8CAdYL0d79.jpg,6.7,5.76,eXistenZ (1999) Tigole 6.2 4500 David Cronenberg
4,Grosse Pointe Blank (1997),George Armitage,5.8,3500,r00t,Grosse Pointe Blank (1997) (1080p BluRay x265 r00t).mkv,F:\1990 Decade\6 - 6.5\16 Grosse Pointe Blank (1997) r00t 6 3500,tt0119229,https://www.criticker.com/film/Grosse-Pointe-Blank/,Grosse Pointe Blank,9434.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//7lQ0MSNQqBTYpXTTZL82u1n95Z3.jpg,5.3,6.38,Grosse Pointe Blank (1997) r00t 5.8 3500 George Armitage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,Princess Mononoke (1997),Hayao Miyazaki,7.9,8750,Tigole,Princess Mononoke 1997 HMC (1080p x265 10bit Tigole).mkv,F:\Director\Hayao Miyazaki\Princess Mononoke (1997) Tigole 8.3,tt0119698,https://www.criticker.com/film/Princess-Mononoke/,Princess Mononoke,128.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//cMYCDADoLKLbB83g4WnJegaZimC.jpg,7.9,7.83,Princess Mononoke (1997) Tigole 7.9 8750 Hayao Miyazaki
702,Code Unknown (2000),Michael Haneke,7.3,750,Tigole,Code Unknown 2000 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Michael Haneke\Code Unknown (2000) Tigole 7.4,tt0216625,https://www.criticker.com/film/Code-Unknown/,Code Unknown,30970.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//kJJx2FNmiaep8WT21uMPY9qPzxL.jpg,8.0,6.68,Code Unknown (2000) Tigole 7.3 750 Michael Haneke
703,Topsy-Turvy (1999),Mike Leigh,6.7,500,Tigole,Topsy-Turvy 1999 Criterion (1080p x265 10bit Tigole).mkv,F:\Director\Mike Leigh\Topsy Turvy (1999) Tigole 6.7,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,Topsy-Turvy (1999) Tigole 6.7 500 Mike Leigh
704,"Ming-liang Tsai, (2005)",Mike Leigh,6.7,500,NaN,"Tian bian yi duo yun (Ming-liang Tsai, 2005).mkv",F:\Director\Ming-liang Tsai 90\The Wayward Cloud (2005) YTS 7.2 400,tt0151568,https://www.criticker.com/film/Topsy-Turvy/,Topsy-Turvy,46435.0,https://image.tmdb.org/t/p/w600_and_h900_bestv2//vDYy9GMV6U5HJOG3WTAU7ACmkmr.jpg,6.6,6.78,"Ming-liang Tsai, (2005) nan 6.7 500 Mike Leigh"


In [137]:
df.Director.value_counts()

Martin Scorsese            21
Lars von Trier             19
Abbas Kiarostami           15
Pedro Almodóvar            15
Ken Loach                  14
Zhang Yimou                13
Joel Coen                  13
Hayao Miyazaki             11
Mike Leigh                 11
Aleksandr Sokurov          11
Kim Ki-duk                 11
Steven Spielberg           10
David Lynch                10
Michael Haneke             10
Hou Hsiao-hsien             9
Claire Denis                9
Wong Kar-wai                8
Jim Jarmusch                8
Ridley Scott                8
Aki Kaurismäki              7
Giuseppe Tornatore          7
Nanni Moretti               7
Tsai Ming-liang             7
Gus Van Sant                7
Michael Mann                6
Dariush Mehrjui             6
Clint Eastwood              6
Danny Boyle                 6
Béla Tarr                   6
Tony Gatlif                 5
Jean-Pierre Jeunet          5
Ang Lee                     5
Zeki Demirkubuz             5
Majid Maji